In [2]:
import openai
openai.api_key = ""

In [3]:
from data.completion_dataset import CompletionMetadata, CompletionDataset
from oai.inference import infer_completion_data
from data.completion_dataset import CompletionIdentifier
from data.split import load_train_test_split 
from evaluation.evaluator import Evaluator
from evaluation.summary import summarize_evaluation 
from oai.finetune import init_finetune, generate_finetune_data_from_completion_dataset
from oai.utils.api_wrapper import fetch_model_ids
from oai.utils.metadata import get_file_id, get_finetune_id, get_model_id, get_model_key

In [2]:

#create a new dataset that only contains the first 7000 samples from multiWD
"""
Finetune = train, train/test split = valiation + test 
multiWD has 7000 samples, train index=0-4899, test index = 4900-6999
Irf has 4928 samples, train = 0-3449,  test = 3450-4927
"""
import json
dataset= "lrf"
file_path = f"/home/azureuser/reasoning-teacher/data/dataset/{dataset}.json"

jsonl_list = []
with open(file_path, "r") as source:
    data = json.load(source)
if dataset == "lrf": n = 3449
if dataset == "multiwd": n = 4900
for i in range(n):
    elem = data['data'][i]
    question =elem['question']
    answer = elem['answer']

    line_dict = {"prompt":question + " You only need to answer yes/no. ###" , "completion": " --> "+answer+" END"}
    jsonl_list.append(line_dict)

# write and save
saved_path = f"/home/azureuser/reasoning-teacher/saved/finetune_data/P_openai/F_zs_cot_{dataset}_baseline.jsonl"
with open(saved_path, 'w') as jsonl_file:
    for item in jsonl_list:
        # Convert the dictionary to a JSON string and write it as a line
        json_line = json.dumps(item)
        jsonl_file.write(json_line + '\n')

print("JSONL data saved to", saved_path)


JSONL data saved to /home/azureuser/reasoning-teacher/saved/finetune_data/P_openai/F_zs_cot_lrf_baseline.jsonl


In [16]:
#revise index of irf
import json

total_samples = 4928
train_percentage = 0.7

split_point = int(total_samples * train_percentage)

data = {
    "train": list(range(0, split_point)),
    "test": list(range(split_point, total_samples))
}

file_path = "/home/azureuser/reasoning-teacher/data/split/lrf__default.json"

with open(file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"JSON file '{file_path}' created successfully.")

JSON file '/home/azureuser/reasoning-teacher/data/split/lrf__default.json' created successfully.


In [17]:
#revise index of multiwd
import json

total_samples = 7000
train_percentage = 0.7

split_point = int(total_samples * train_percentage)

data = {
    "train": list(range(0, split_point)),
    "test": list(range(split_point, total_samples))
}
file_path = "/home/azureuser/reasoning-teacher/data/split/multiwd__default.json"

with open(file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"JSON file '{file_path}' created successfully.")

JSON file '/home/azureuser/reasoning-teacher/data/split/multiwd__default.json' created successfully.


In [28]:
# dataset_key = "lrf" 
dataset_key = "multiwd" 
#酌情更改
finetune_key = "zs_cot_{}_baseline".format(dataset_key)
# train_key = "ft_cot_baseline"
train_key = "ft_cot_baseline10" #need to finetune new models for Lrf

In [11]:
import json
from paths import get_finetune_data_path
with open(get_finetune_data_path("openai", finetune_key)) as f:
    print(json.dumps(json.loads(f.readline()), indent=4))

{
    "prompt": "All my life i've been going through shit (only 17 years old) and when things started to get better i crashed. I can't get myself to get out of bed no matter how much i try, my family understands but do still not approve since my grades dropped from all A's to E-C. It has been like this for 1-2 years now and none of my friends understands how It's like, I can't really blame them either since I don't like talking about it and i've always been taught to be a man and keep this stuff to myself. They just see a lazy fuck who is too irresponsible to go too school, same with my teachers. Idk if typing here is going to help at all but if anyone has some tips/advice on how to get motivated again i would be super happy. Does the post shows risk of thwarted belongingness? ###",
    "completion": " --> yes END"
}


In [29]:
# train_key = "multiwd" #made a mistake, trained ada,babbage and curie of multiwd with this train key
student_model = "ada"
# student_model = "babbage"
# student_model = "curie"

In [40]:
# Note, train_key identifies the method used to train the model, i.e., the method used to fine-tune the base model.
init_finetune(finetune_key, student_model, dataset_key, train_key)

Created OpenAI finetune `B_babbage__D_multiwd__T_ft_cot_baseline10`: `ft-EzuUiF5QQhIHfdMIGWmoPSEO`


'B_babbage__D_multiwd__T_ft_cot_baseline10'

In [16]:
fetch_model_ids()

Fetching model ids from 7 finetunes
----------------------------------------------------------------------------------------------------
model_key                                                                       status              
----------------------------------------------------------------------------------------------------
B_curie__D_multiwd__T_multiwd                                                   failed              
B_curie__D_lrf__T_ft_cot_baseline                                               failed              
B_ada__D_lrf__T_ft_cot_baseline2                                                failed              
B_curie__D_lrf__T_ft_cot_baseline2                                              failed              
B_ada__D_lrf__T_ft_cot_baseline3                                                failed              
B_ada__D_lrf__T_ft_cot_baseline4                                                failed              
B_curie__D_lrf__T_IRF_16_Shots                         

False

In [30]:
model_key = get_model_key(student_model, dataset_key, train_key)
model_key

'B_ada__D_multiwd__T_ft_cot_baseline10'

In [ ]:
model_keys_running =[
    "B_curie__D_multiwd__T_multiwd",
    "B_curie__D_lrf__T_ft_cot_baseline",
]
model_key_finished= [
    "B_ada__D_lrf__T_ft_cot_baseline",
    "B_babbage__D_lrf__T_ft_cot_baseline",
    "B_ada__D_multiwd__T_ft_cot_baseline",
    "B_babbage__D_multiwd__T_ft_cot_baseline",
]

## Infer Student Model

In [31]:
# Note, completion_key and train_key are both "ft_cot_test". Recall that completion_key refers to
# the method used to generate completions by the student model, and train_key refers to the method
# used to train the student model.
completion_metadata = CompletionMetadata(base_model=student_model, completion_key="zs",
                                         dataset_key=dataset_key, finetune_key=finetune_key,
                                         prediction_template="ft_cot_token",
                                         train_key=train_key, epoch=None)
train, test = load_train_test_split(dataset_key)
# 这一步是通过index去split的，
# lrf train: sample 0-3942, test: 3942-4927
# multiwd train: sample 0-5600 test:5601-7000

In [32]:
# Note, `infer_completion_data` will find our new student model (that we fetched above) by using
#       `base_model`, `dataset_key`, and `train_key` which is specified in `completion_metadata`.
completion_dataset = infer_completion_data(completion_metadata, zs_cot_step=None,
                                           sample_indices=test, augs=1, temperature=0,
                                           max_tokens=512)  # note, we use 1024 tokens for student inference

Initializing new CompletionDataset at:
/home/azureuser/reasoning-teacher/saved/completion_data/B_ada__C_zs/D_multiwd__T_ft_cot_baseline10.json
Inferring completions for 2100 remaining samples (total=2100)


Inferring completions via OpenAI: 100%|██████████| 2100/2100 [07:10<00:00,  4.88it/s]


## Evaluate on Student Model

In [33]:
completion_identifier = CompletionIdentifier(student_model, completion_key="zs", dataset_key=dataset_key,
                                             train_key=train_key)
completion_dataset = CompletionDataset.load(completion_identifier)
train, test = load_train_test_split(dataset_key)

In [34]:
evaluator = Evaluator(dataset_key, "zs")
evaluation = evaluator.evaluate_completion_dataset(completion_dataset, test)

In [35]:
summarize_evaluation(evaluation)

{'accuracy': 0.8738095238095238,
 'contains_answer': 0.960952380952381,
 'correct_format': 0.0,
 'complete': 0.0}

In [19]:
#test the evaluate function 
evaluator = Evaluator(dataset_key, "zs")
comp = {
                "sample_index": 3944,
                "completion_index": 0,
                "question": "I feel like life is too hard but I' not sure whether to kill myself. I've been to a mental hospital for appointments and medication but I want to be sectioned instead of forced to live in the normal everyday life that everyone else is living. Anyone know how I can get in? Does the post shows risk of perceived burdensomeness?",
                "answer": "yes",
                "prompt": "I feel like life is too hard but I' not sure whether to kill myself. I've been to a mental hospital for appointments and medication but I want to be sectioned instead of forced to live in the normal everyday life that everyone else is living. Anyone know how I can get in? Does the post shows risk of perceived burdensomeness? ###",
                "completion": "yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post shows risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post shows risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###yes, the post does show risk of perceived burdensomeness, but I'm not sure if it's worth the risk. Does the post does show risk of perceived burdensomeness? ###",
                "finish_reason": "length"
            }
evaluator.evaluate_completion(comp)

{'sample_index': 3944,
 'completion_index': 0,
 'correct': True,
 'contains_answer': True,
 'correct_format': True,
 'complete': False}

## TEST

### 1) create finetune file

In [116]:
from paths import get_finetune_data_path, SAVED_PATH
from oai.utils.metadata import get_file_id, set_file_id, get_finetune_id, set_finetune_id


def create_finetune_file(daset_key):
    #get path 
    test_finetune_key = "zs_cot_{}_baseline".format(dataset_key)
    path = get_finetune_data_path("openai", test_finetune_key)

    #create finetune path in openai
    with open(path) as f:
        response = openai.File.create(
            file=f,
            purpose='fine-tune'
        )

    #it will save the finetuned file in local
    file_id = response["id"]
    set_file_id(test_finetune_key, file_id)
    print("Created OpenAI File for `{}`: `{}`".format(test_finetune_key, response["id"]))
    return file_id

In [118]:
#Revise here
dataset_key = "multiwd"
file_id = create_finetune_file(dataset_key)

Created OpenAI File for `zs_cot_multiwd_baseline`: `file-yK38WJ1deWtuogwxV5bxseAs`


In [ ]:
"Created OpenAI File for `zs_cot_lrf_baseline`: `file-WfPIBUuHgNWP10W8FXN0soKq`"

### 2) openai finetune

In [120]:
model_key = get_model_key(student_model, dataset_key, train_key)
model_key

'B_babbage__D_multiwd__T_test2'

In [33]:
def finetune(train_key, student_model,dataset_key, file_id):
    response = openai.FineTune.create(training_file=file_id, model=student_model)
    finetune_id = response["id"]

    model_key = get_model_key(student_model, dataset_key, train_key)
    set_finetune_id(model_key, finetune_id)
    print("Created OpenAI finetune `{}`: `{}`".format(model_key, finetune_id))

In [119]:
#Revise here
# student_model = "ada"
student_model = "babbage"
# student_model = "curie"
train_key = "test2" 
finetune(train_key, student_model,dataset_key)

Created OpenAI finetune `B_babbage__D_multiwd__T_test2`: `ft-fw5onuw3h1AqMGg0MwxU4JeK`


In [131]:
fetch_model_ids()

Fetching model ids from 10 finetunes
----------------------------------------------------------------------------------------------------
model_key                                                                       status              
----------------------------------------------------------------------------------------------------
B_curie__D_multiwd__T_multiwd                                                   failed              
B_curie__D_lrf__T_ft_cot_baseline                                               failed              
B_ada__D_lrf__T_ft_cot_baseline2                                                failed              
B_curie__D_lrf__T_ft_cot_baseline2                                              failed              
B_ada__D_lrf__T_ft_cot_baseline3                                                failed              
B_ada__D_lrf__T_ft_cot_baseline4                                                failed              
B_curie__D_lrf__T_IRF_16_Shots                        

False

### 3) Infer student model

In [73]:
# create test data

"""
Finetune = train, train/test split = valiation + test 
multiWD has 7000 samples, train index=0-4899, test index = 4900-6999
Irf has 4928 samples, train = 0-3449,  test = 3450-4927
"""
import json 
def create_testset(dataset):

    file_path = f"/home/azureuser/reasoning-teacher/data/dataset/{dataset}.json"

    jsonl_list = []
    with open(file_path, "r") as source:
        data = json.load(source)
    if dataset == "lrf": n = 1478
    if dataset == "multiwd": n = 2100
    for i in range(n - 1, -1, -1):
        elem = data['data'][i]
        question =elem['question']
        answer = elem['answer']

        line_dict = {"prompt":question + " You only need to answer yes/no. ###" , "answer": " --> "+answer+" END"}
        jsonl_list.append(line_dict)
    
    return jsonl_list
# write and save
# saved_path = f"/home/azureuser/reasoning-teacher/saved/finetune_data/P_openai/F_zs_cot_{dataset}_baseline.jsonl"
# with open(saved_path, 'w') as jsonl_file:
# for item in jsonl_list:
#     # Convert the dictionary to a JSON string and write it as a line
#     json_line = json.dumps(item)


In [123]:
len(jsonl_list)

1478

In [130]:
dataset= "multiwd" ## revise
jsonl_list = create_testset(dataset)
jsonl_list[0]

{'prompt': "I've been dealing with depression for five years and I'm honestly sick of the crappy grades I get. I'm currently in high school and I just want to get better grades, but dealing with mental illness makes it seem impossible Does the emotional wellness dimension exists in the post? You only need to answer yes/no. ###",
 'answer': ' --> yes END'}

In [129]:
model_key = get_model_key(student_model, dataset_key, train_key)
model_key

'B_ada__D_multiwd__T_test2'

In [132]:
from oai.utils.metadata import get_model_key, get_model_id 

#This step will get answers!!!!!

model_id = get_model_id(model_key) #it will change everytime

for i in range(len(jsonl_list)):
    prompt = jsonl_list[i]['prompt']

    response = openai.Completion.create(model=model_id, prompt=prompt, max_tokens=2,
                                         temperature=0.0, n=1)
    res = response['choices'][0]['text']
    prediction = res.split('--> ')[-1]
    jsonl_list[i]['prediction'] = prediction

In [34]:
import json
answer = response['choices'][0]['text']
answer

' --> yes'

In [93]:
def eval(jsonl_list):
    correct = 0
    for elem in jsonl_list:
        answer = elem['answer'].split('--> ')[-1].split(' END')[0]
        prediction = elem['prediction']
        if answer == prediction:
            correct +=1
        
    acc = correct/ len(jsonl_list)
    return acc

In [134]:
jsonl_list

[{'prompt': "I've been dealing with depression for five years and I'm honestly sick of the crappy grades I get. I'm currently in high school and I just want to get better grades, but dealing with mental illness makes it seem impossible Does the emotional wellness dimension exists in the post? You only need to answer yes/no. ###",
  'answer': ' --> yes END',
  'prediction': 'yes'},
 {'prompt': "I've been dealing with depression for five years and I'm honestly sick of the crappy grades I get. I'm currently in high school and I just want to get better grades, but dealing with mental illness makes it seem impossible Does the vocational wellness dimension exists in the post? You only need to answer yes/no. ###",
  'answer': ' --> no END',
  'prediction': 'yes'},
 {'prompt': "I've been dealing with depression for five years and I'm honestly sick of the crappy grades I get. I'm currently in high school and I just want to get better grades, but dealing with mental illness makes it seem impossi

In [133]:
eval(jsonl_list)

0.9619047619047619

In [135]:
{"lrf_babbage":0.9810554803788903,
 "lrf_ada": 0.9790257104194858,
 "multiwd_ada": 0.9619047619047619}

{'lrf_babbage': 0.9810554803788903,
 'lrf_ada': 0.9790257104194858,
 'multiwd_ada': 0.9619047619047619}

In [ ]:
: